# Functional API Preview

This notebook provides a preview of the proposed functional API for dowhy. For details on the new API for DoWhy, check out https://github.com/py-why/dowhy/wiki/API-proposal-for-v1 It is a work-in-progress and is updated as we add new functionality. We welcome your feedback through Discord or on the Discussions page.
This functional API is designed with backwards compatibility. So both the old and new API will continue to co-exist and work for the immediate new releases. Gradually the old API using CausalModel will be deprecated in favor of the new API.

### Setup data for tests

In [ ]:
import dowhy as dw
from dowhy.causal_identifier import (
    AutoIdentifier,
    BackdoorAdjustment,
    IDIdentifier,
)

In [ ]:
from dowhy.causal_graph import CausalGraph
import dowhy.datasets

data = dowhy.datasets.linear_dataset(
    beta=4,
    num_common_causes=2,
    num_instruments=1,
    num_effect_modifiers=1,
    num_samples=500,
    treatment_is_binary=True,
    stddev_treatment_noise=10,
    num_discrete_common_causes=1,
)

In [ ]:
treatment_name = data["treatment_name"]
outcome_name = data["outcome_name"]

graph = CausalGraph(
    treatment_name=treatment_name,
    outcome_name=outcome_name,
    graph=data["gml_graph"],
    effect_modifier_names=data["effect_modifier_names"],
    common_cause_names=data["common_causes_names"],
    observed_node_names=data["df"].columns.tolist(),
)

### Functional API Usage

This section describes how to use the **experimental** functional API

In [ ]:
# New functional API identify_effect function this calls auto_identify_effect with pre-set parameters

identified_estimand_auto = dw.identify_effect(graph=graph, treatment_name=treatment_name, outcome_name=outcome_name)
print(identified_estimand_auto)

In [ ]:
# New functional API (AutoIdentifier with default parameters backdoor_adjustment = DEFAULT)

identified_estimand_auto = dw.auto_identify_effect(
    graph=graph, treatment_name=treatment_name, outcome_name=outcome_name, estimand_type=dw.EstimandType.NONPARAMETRIC_ATE
)
print(identified_estimand_auto)

In [ ]:
# New functional API (AutoIdentifier with custom parameters)

identified_estimand_auto = dw.auto_identify_effect(
    graph=graph,
    treatment_name=treatment_name,
    outcome_name=outcome_name,
    estimand_type=dw.EstimandType.NONPARAMETRIC_NIE,
    backdoor_adjustment=BackdoorAdjustment.BACKDOOR_MIN_EFFICIENT,
)
print(identified_estimand_auto)

In [ ]:
# New functional API (IDIdentifier)
identified_estimand_id = dw.id_identify_effect(
    graph=graph,
    treatment_name=treatment_name,
    outcome_name=outcome_name,
)
print(identified_estimand_id)

### Backwards compatibility

This section shows the same results as the new Functional API with the old `CausalModel` API

In [ ]:
# CausalModel API with AutoIdentifier

model = dw.CausalModel(
    data=data["df"],
    treatment=data["treatment_name"],
    outcome=data["outcome_name"],
    graph=data["gml_graph"],
)

identified_estimand = model.identify_effect()
print(identified_estimand)

In [ ]:
# CausalModel API with IDIdentifier 

model = dw.CausalModel(
    data=data["df"],
    treatment=data["treatment_name"],
    outcome=data["outcome_name"],
    graph=data["gml_graph"],
)

identified_estimand = model.identify_effect(method_name="id-algorithm")
print(identified_estimand)